## Autotune OntoVAE models

The OntoVAE package contains a class ModelTuner() to do autotuning of the hyperparameters of OntoVAE. This vignette shows how to utilize this class to get optimal hyperparameter settings.

### Creating a Tuner for OntoVAEcpa

In [ ]:
import sys
sys.path.append('/workspace')
import scanpy as sc
from cobra_ai.module.ontobj import *
from cobra_ai.module.utils import *
from cobra_ai.model.cobra import *
from cobra_ai.module.autotune import *
from ray import tune

In [ ]:
# load ontobj
ontobj = Ontobj()
ontobj.load('cobra_ai/GO/GO.ontobj')

# load anndata
adata = sc.read_h5ad('cobra_ai/data/train_pbmc.h5ad')

# setup the anndata
cobra_keys = ['n_genes']
adata = utils.setup_anndata_ontovae(adata, ontobj, cobra_keys = cobra_keys)


In [ ]:
# create the tuner and show a summary
tuner = ModelTuner(COBRA)
tuner.info()

### Tune Hyperparameters

In [ ]:
# create the search space dictionary with the parameters you want to define a search space for yourself
search_space = {"hidden_layers_class": tune.choice([1, 2]), "lr_vae": tune.loguniform(1e-4, 1e-2)}
# create a list of parameters that you also want to tune with the default search space specified in the package
use_defaults = ['drop_class', 'kl_coeff']

You can have a look at the default search space with the following code snippet:

In [ ]:
DEFAULTS[COBRA.__name__]
DEFAULTS[OntoVAE.__name__]

In [ ]:
# run the fit function of the tuner
results = tuner.fit(adata, ontobj, search_space = search_space, use_defaults = use_defaults, epochs = 100, cobra_keys = cobra_keys, num_samples = 10, resources = {'gpu': 1})
# show the best hyperparameter settings
tuner.output(results[0], results[1])